In [3]:
import nbformat

# Path to this notebook in Colab
notebook_path = '/content/LawGorithmML/ML/Untitled25.ipynb'

# Load the current notebook
with open('/content/LawGorithmML/ML/Untitled25.ipynb', 'r') as f:  # replace with your notebook's actual name
    nb = nbformat.read(f, as_version=nbformat.NO_CONVERT)

# Remove problematic widget metadata
if 'widgets' in nb.metadata:
    del nb.metadata['widgets']

# Save cleaned version
with open(notebook_path, 'w') as f:
    nbformat.write(nb, f)

print(f"Cleaned notebook saved to {notebook_path}")


Cleaned notebook saved to /content/LawGorithmML/ML/Untitled25.ipynb


In [2]:
!git clone https://github.com/RaphaelEse/LawGorithmML.git

Cloning into 'LawGorithmML'...
remote: Enumerating objects: 837, done.
remote: Counting objects: 100% (161/161), done.
remote: Compressing objects: 100% (141/141), done.
remote: Total 837 (delta 38), reused 121 (delta 18), pack-reused 676 (from 1)
Receiving objects: 100% (837/837), 56.18 MiB | 19.06 MiB/s, done.
Resolving deltas: 100% (377/377), done.


In [ ]:
!pip install datasets

In [ ]:
    from google.colab import drive
    drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
drive_save_path = "/content/drive/My Drive/legal_nlp_artifacts"
os.makedirs(drive_save_path, exist_ok=True)


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
import torch
from datasets import Dataset
import os
import glob
import pandas as pd
import json
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np
import concurrent.futures
from sklearn.utils import resample
import nltk
from nltk.tokenize import sent_tokenize
from concurrent.futures import ThreadPoolExecutor
from IPython.display import display

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
nltk.download("punkt")
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
# Define the path to the full-text bills
bills_path = "/content/LawGorithmML/citations/citations/sample_bills"

# Get all text files in the directory
bill_files = sorted(glob.glob(os.path.join(bills_path, "*.txt")))

bill_texts = {}
def load_file(file):
    with open(file, "r", encoding="utf-8") as f:
        return os.path.basename(file), f.read()

# Load 108 bills using multi-threading
with concurrent.futures.ThreadPoolExecutor() as executor:
    results = executor.map(load_file, bill_files)

bill_texts = dict(results)

print(f"Loaded {len(bill_texts)} bills.")

Loaded 108 bills.


In [ ]:
json_path = "/content/LawGorithmML/labeled_citations_zero_shot/"
# Get all JSON files in the directory
# Load cleaned citation JSON files
json_files = sorted(os.listdir(json_path))

# Read and combine JSON files
cleaned_citations = []
for file in json_files:
    with open(os.path.join(json_path, file), "r", encoding="utf-8") as f:
        cleaned_citations.extend(json.load(f))

# Convert to DataFrame
df_citations = pd.DataFrame(cleaned_citations)

df_citations["normCite"] = df_citations["normCite"].astype(str)

# Filter out citations without `normCite`
df_citations = df_citations[df_citations["normCite"].notna()]
df_citations = df_citations[df_citations["normCite"].str.strip() != ""]
# Display dataset size
print(f"Loaded {len(json_files)} cleaned citation files with {len(df_citations)} citations.")
df_citations.head()


Loaded 77 cleaned citation files with 17405 citations.


,text,startPosition,endPosition,normCite,citeType,altCite,pinCiteStr,pageRangeStr,nodeId,section,sectionAndSubSection,isShortCite,chunk_id,high_confidence
13,Sec. 604,6331,6339,33 usc 1384,USC,33 usc 1384,,,0,,,True,0.0,Precedent
31,Sec. 104,9376,9384,42 usc 5304,USC,42 usc 5304,,,0,,,True,0.0,Amendment
32,Sec. 105,9480,9488,42 usc 5305,USC,42 usc 5305,,,0,,,True,0.0,Example
44,Sec. 204,10663,10671,7 usc 7508,USC,7 usc 7508,,,0,,,True,0.0,Amendment
48,Sec. 211,11303,11311,34 usc 60531,USC,34 usc 60531,,,0,,,True,0.0,Amendment


In [ ]:
# Count occurrences of each string label
print(df_citations["high_confidence"].value_counts())

high_confidence
Example       6612
Amendment     3962
Precedent     2556
Authority     1651
Exception     1425
Definition    1199
Name: count, dtype: int64


In [ ]:
bill_sentences = {
    name: sent_tokenize(text)
    for name, text in bill_texts.items()
}

def extract_context(row):
    #Extracts context from a bill given a citation
    citation_text = row["text"]

    for bill_name, sentences in bill_sentences.items():
        for i, sentence in enumerate(sentences):
            if citation_text in sentence:
                start = max(0, i - 2)
                end = i + 1  # Only before and including citation sentence
                selected = " ".join(sentences[start:end]).strip()
                selected = re.sub(r"\s+", " ", selected)  # Clean up spaces
                return selected if len(selected) > 20 else None
    return None

# Convert to list for parallel processing
rows = df_citations.to_dict(orient="records")


with ThreadPoolExecutor() as executor:
    contexts = list(executor.map(extract_context, rows))

# Assign back to DataFrame
df_citations["context"] = contexts

# Drop rows with no context
df_citations = df_citations.dropna(subset=["context"])

# View result
df_citations[["text", "context"]].head()


,text,context
113,section 101(9),REFERENCES Except as expressly provided otherw...
120,section 101,REFERENCES Except as expressly provided otherw...
121,section 101,REFERENCES Except as expressly provided otherw...
123,section 101,REFERENCES Except as expressly provided otherw...
124,section 101,REFERENCES Except as expressly provided otherw...


In [ ]:
# View result
df_citations[["text", "context"]].head(10)

,text,context
113,section 101(9),REFERENCES Except as expressly provided otherw...
120,section 101,REFERENCES Except as expressly provided otherw...
121,section 101,REFERENCES Except as expressly provided otherw...
123,section 101,REFERENCES Except as expressly provided otherw...
124,section 101,REFERENCES Except as expressly provided otherw...
125,section 101,REFERENCES Except as expressly provided otherw...
126,section 251(b)(2)(A)(i) of the Balanced Budget...,Amounts made available by section 101 for Depa...
128,section 101,REFERENCES Except as expressly provided otherw...
130,section 101,REFERENCES Except as expressly provided otherw...
132,section 101,REFERENCES Except as expressly provided otherw...


In [ ]:
# Keep only necessary columns
df = df_citations[["text", "normCite", "citeType", "context", "high_confidence"]]

# Rename columns for clarity
df = df.rename(columns={"high_confidence": "label"})

# Encode labels as numbers (since models need numerical labels)
label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["label"])

# Save label mapping for later decoding
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Label Mapping:", label_mapping)

# Split into train/test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df["context"], df["label"], test_size=0.2, random_state=42
)

# Convert into Pandas DataFrame format
train_df = pd.DataFrame({"text": train_texts, "label": train_labels})
test_df = pd.DataFrame({"text": test_texts, "label": test_labels})

# Ensure training and test dataframes have contexts and labels
print(f"Training Samples: {len(train_df)}, Test Samples: {len(test_df)}")
train_df.head(10)

Label Mapping: {'Amendment': np.int64(0), 'Authority': np.int64(1), 'Definition': np.int64(2), 'Example': np.int64(3), 'Exception': np.int64(4), 'Precedent': np.int64(5)}
Training Samples: 7588, Test Samples: 1898


,text,label
6564,"9011)), cotton, and sugar; and (ii) shall not ...",3
22688,"9011)), cotton, and sugar; and (ii) shall not ...",2
21605,BANNING OF PRODUCTS CONTAINING A HIGH CONCENTR...,3
1896,Domestic Food Programs Food and Nutrition Serv...,1
1833,Domestic Food Programs Food and Nutrition Serv...,2
23389,"9011)), cotton, and sugar; and (ii) shall not ...",0
7955,"9011)), cotton, and sugar; and (ii) shall not ...",5
5019,Domestic Food Programs Food and Nutrition Serv...,3
264,Amounts made available by section 101 for Depa...,5
4605,BANNING OF PRODUCTS CONTAINING A HIGH CONCENTR...,5


In [ ]:
# LegalBERT
MODEL_NAME = "nlpaueb/legal-bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Convert Pandas DF to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Tokenize dataset
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

# Load pre-trained model for classification
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=len(label_encoder.classes_))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/222k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Map:   0%|          | 0/7588 [00:00<?, ? examples/s]

Map:   0%|          | 0/1898 [00:00<?, ? examples/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nlpaueb/legal-bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f" Using device: {device}")

 Using device: cuda


In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="weighted")
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
model.to(device)
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=50,
    weight_decay=0.05,
    fp16=True,
    logging_dir='./logs',
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics
)
# Train the model
trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,1.518101,0.425184,0.321399,0.361588,0.425184
2,No log,1.511408,0.431507,0.339237,0.396970,0.431507
3,1.485000,1.502697,0.435722,0.346856,0.409021,0.435722
4,1.485000,1.511055,0.431507,0.352914,0.384591,0.431507
5,1.454800,1.515994,0.432561,0.351335,0.395536,0.432561
6,1.454800,1.499831,0.436249,0.358616,0.416307,0.436249
7,1.417000,1.501251,0.437829,0.371744,0.431516,0.437829
8,1.417000,1.511987,0.429926,0.372113,0.428139,0.429926
9,1.369900,1.527329,0.440464,0.376223,0.441155,0.440464
10,1.369900,1.522360,0.437302,0.372375,0.435189,0.437302


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# Save model and tokenizer
model.save_pretrained("./fine_tuned_legalbert")
tokenizer.save_pretrained("./fine_tuned_legalbert")
# Zip the model folder
!zip -r fine_tuned_legalbert.zip fine_tuned_legalbert

# Download to local computer
from google.colab import files
files.download("fine_tuned_legalbert.zip")


  adding: fine_tuned_legalbert/ (stored 0%)
  adding: fine_tuned_legalbert/model.safetensors (deflated 7%)
  adding: fine_tuned_legalbert/vocab.txt (deflated 51%)
  adding: fine_tuned_legalbert/special_tokens_map.json (deflated 42%)
  adding: fine_tuned_legalbert/tokenizer_config.json (deflated 75%)
  adding: fine_tuned_legalbert/tokenizer.json (deflated 71%)
  adding: fine_tuned_legalbert/config.json (deflated 55%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#Retreive and use model
model = AutoModelForSequenceClassification.from_pretrained("./fine_tuned_legalbert")
tokenizer = AutoTokenizer.from_pretrained("./fine_tuned_legalbert")

In [ ]:
demo_samples = df_citations.sample(5)["context"].tolist()

In [ ]:
lTrain_df = train_df.sample(frac=0.25, random_state=42).reset_index(drop=True)

# Reduce the test set for longformer model
lTest_df = test_df.sample(frac=0.25, random_state=42).reset_index(drop=True)

# Display new sizes
print(f"🔹 New training set size: {len(lTrain_df)}")
print(f"🔹 New test set size: {len(lTest_df)}")


🔹 New training set size: 3330
🔹 New test set size: 833


In [ ]:
#Longformer
MODEL_NAME = "allenai/longformer-base-4096"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=4096,
        return_tensors="pt"
    )

# Convert Pandas DF to Hugging Face Dataset
train_dataset = Dataset.from_pandas(lTrain_df)
test_dataset = Dataset.from_pandas(lTest_df)

# Tokenize dataset
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

# Load pre-trained model for classification
lModel = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=len(label_encoder.classes_))

Map:   0%|          | 0/3330 [00:00<?, ? examples/s]

Map:   0%|          | 0/833 [00:00<?, ? examples/s]

Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
lModel.to(device)
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=7,
    weight_decay=0.05,
    fp16=True,
    logging_dir='./logs',
)

# Trainer
trainer = Trainer(
    model=lModel,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics
)
# Train the model
trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,1.576200,1.561742,0.441777,0.328123,0.285628,0.441777
2,1.555900,1.540927,0.445378,0.334423,0.289112,0.445378


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# Save model and tokenizer
lModel.save_pretrained("./fine_tuned_longformer")
tokenizer.save_pretrained("./fine_tuned_longformer")
# Download to local computer
from google.colab import files
files.download("fine_tuned_longformer.zip")